# HDS5210-2024 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 24th.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [1]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status):
  """
  the priest() calculates the clinical severity score, the 30-day probability of an outcome as a risk percentage.

  The function takes several input parameters and calculates a score:

  Sex (Gender assigned at birth)
  Age in years
  Respiratory rate in breaths per minute
  Oxygen saturation as a percent between 0 and 1
  Heart rate in beats per minute
  Systolic BP in mmHg
  Temperature in degrees C
  Alertness as a string description
  Inspired Oxygen as as string description
  Performance Status as a string description

  Based on this score, the risk percentage is calculated. Risk percentage is returned as a float between 0 and 1.

  >>> priest('male',67,10,0.95,41,91,35.1,'ALERT','AIR','UNRESTRICTED NORMAL ACTIVITY') == 0.29
  True

  >>> priest('Female',21,19,0.95,67,111,36.1,'ALERT','AIR','UNRESTRICTED NORMAL ACTIVITY') == 0.01
  True

  >>> priest('MALE',75,24,0.92,39,94,35.1,'ALERT','SUPPLEMENTAL OXYGEN','LIMITED STRENUOUS ACTIVITY, CAN DO LIGHT ACTIVITY') == 0.59
  True
  """

  score = 0
  risk = 0.0
  invalid=False

  if sex.upper() == 'MALE': score+=1
  elif sex.upper() == 'FEMALE': score+=0
  else: invalid=True

  if 16<=age<=49: score += 0
  elif 50<=age<=65: score += 2
  elif 66<=age<=80: score += 3
  elif age >80: score += 4
  else: invalid=True

  if respiratory_rate <9: score += 3
  elif 9<=respiratory_rate<=11: score += 1
  elif 12<=respiratory_rate<=20: score +=0
  elif 21<=respiratory_rate<=24: score +=2
  elif respiratory_rate>24: score += 3
  else: invalid=True

  if oxygen_saturation > 0.95: score += 0
  elif 0.94 <= oxygen_saturation <= 0.95: score += 1
  elif 0.92 <= oxygen_saturation <= 0.93: score += 2
  elif oxygen_saturation<= 0.92: score += 3
  else: invalid=True

  if heart_rate < 41: score += 3
  elif 41 <= heart_rate <= 50: score += 1
  elif 51 <= heart_rate <= 90: score += 0
  elif 91 <= heart_rate <= 110: score += 1
  elif 111 <= heart_rate <= 130: score += 2
  elif heart_rate > 130: score += 3
  else: invalid=True

  if systolic_bp < 91: score += 3
  elif 91 <= systolic_bp <= 100: score += 2
  elif 101 <= systolic_bp <= 110: score += 1
  elif 111 <= systolic_bp <= 219: score += 0
  elif systolic_bp > 219: score += 3
  else: invalid=True

  if temperature < 35.1: score += 3
  elif 35.1 <= temperature <= 36.0: score += 1
  elif 36.1 <= temperature <= 38.0: score += 0
  elif 38.1 <= temperature <= 39.0: score += 1
  elif temperature > 39.0: score += 2
  else: invalid=True

  if alertness.upper()=="ALERT": score+=0
  elif alertness.upper()=="CONFUSED OR NOT ALERT": score+=3
  else: invalid=True

  if inspired_oxygen.upper() == 'AIR': score += 0
  elif inspired_oxygen.upper() == 'SUPPLEMENTAL OXYGEN': score +=2
  else: invalid=True

  if performance_status.upper() == 'UNRESTRICTED NORMAL ACTIVITY': score += 0
  elif performance_status.upper() == 'LIMITED STRENUOUS ACTIVITY, CAN DO LIGHT ACTIVITY': score += 1
  elif performance_status.upper() == 'LIMITED ACTIVITY, CAN SELF-CARE': score += 2
  elif performance_status.upper() == 'LIMITED SELF-CARE': score += 3
  elif performance_status.upper() == 'BED/CHAIR BOUND, NO SELF-CARE': score += 4
  else: invalid=True

  if invalid==True:
    return None
  else:
    if score <= 1: risk = 0.01
    elif 2 <= score <= 3: risk = 0.02
    elif score == 4: risk = 0.03
    elif score == 5: risk = 0.09
    elif score == 6: risk = 0.15
    elif score == 7: risk = 0.18
    elif score == 8: risk = 0.22
    elif score == 9: risk = 0.26
    elif score == 10: risk = 0.29
    elif score == 11: risk = 0.34
    elif score == 12: risk = 0.38
    elif score == 13: risk = 0.46
    elif score == 14: risk = 0.47
    elif score == 15: risk = 0.49
    elif score == 16: risk = 0.55
    elif 17 <= score <= 25: risk = 0.59
    elif score >= 26: risk = 0.99

    return risk





In [2]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/doctest.py", line 1501, in run
    sys.settrace(save_trace)



Finding tests in NoName
Trying:
    priest('male',67,10,0.95,41,91,35.1,'ALERT','AIR','UNRESTRICTED NORMAL ACTIVITY') == 0.29
Expecting:
    True
ok
Trying:
    priest('Female',21,19,0.95,67,111,36.1,'ALERT','AIR','UNRESTRICTED NORMAL ACTIVITY') == 0.01
Expecting:
    True
ok
Trying:
    priest('MALE',75,24,0.92,39,94,35.1,'ALERT','SUPPLEMENTAL OXYGEN','LIMITED STRENUOUS ACTIVITY, CAN DO LIGHT ACTIVITY') == 0.59
Expecting:
    True
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [3]:
import requests
import json
def find_hospital(age, sex, risk):
  """
  The find_hospital() function sends a request to a REST web service using requests module.
  The function takes age, sex and risk as arguments and creates a custom url for the web service.
  The output of the request (a json string) is returned.
  If request was unsuccessful, return None.

  >>> find_hospital(55,'female',0.04) == 'Select Specialty Hospital - Northeast Atlanta'
  True
  >>> find_hospital(65,'male',0.29) == 'Wesley Woods Geriatric Hospital'
  True
  >>> find_hospital(21,'male',0.01) == 'Southwest Hospital and Medical Center'
  True


  """

  url = f"https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}"
  results = requests.get(url)
  if results.status_code == 200:
    addresses = json.loads(results.text)
    return addresses['hospital']
  else:
    return None


In [4]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(55,'female',0.04) == 'Select Specialty Hospital - Northeast Atlanta'
Expecting:
    True
ok
Trying:
    find_hospital(65,'male',0.29) == 'Wesley Woods Geriatric Hospital'
Expecting:
    True
ok
Trying:
    find_hospital(21,'male',0.01) == 'Southwest Hospital and Medical Center'
Expecting:
    True
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [5]:
import requests
import json
def get_address(hospital_name):
  """
  This function gets the address of a given hospital based on the hospital name.
  The data for the hospital addresses is requested from a google drive file.

  Arguments: hospital_name ->  a string containing hospital name

  Outputs: hospital address, as a string
           If hospital address not found then None

  >>> get_address('SATILLA PARK HOSPITAL') == '2500 SATILLA PARKWAY'
  True
  >>> get_address('MEDICAL CENTER OF CENTRAL GEORGIA') == '777 HEMLOCK STREET'
  True
  >>> get_address('RIVERWOODS BEHAVIORAL HEALTH SYSTEM') == '223 MEDICAL CENTER DRIVE'
  True

  """
  found_status=0
  json_string = requests.get('https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-')

  if json_string.status_code == 200:
      addresses = json.loads(json_string.text)
      for key,value in addresses.items():
        if key == hospital_name.upper():
          return value["ADDRESS"]
          found_status+=1
      if found_status==0:
        return None
  else:
    return None

In [6]:
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address('SATILLA PARK HOSPITAL') == '2500 SATILLA PARKWAY'
Expecting:
    True
ok
Trying:
    get_address('MEDICAL CENTER OF CENTRAL GEORGIA') == '777 HEMLOCK STREET'
Expecting:
    True
ok
Trying:
    get_address('RIVERWOODS BEHAVIORAL HEALTH SYSTEM') == '223 MEDICAL CENTER DRIVE'
Expecting:
    True
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [13]:
import requests
import csv
def process_people(file_location):
  """
  This function
  """
  #request psv file and load it as a list
  psv_file = requests.get(file_location)

  if psv_file.status_code == 200:
    psv_text = psv_file.text

    csv_reader = csv.reader(psv_text.splitlines(), delimiter='|')
    people = list(csv_reader)
    #need to remove headers, which is the first element of the list
    people.pop(0)

    #create a dictionary based on psv list, where first element is the key
    people_dict={}
    for person in people:
    #get all individual variables to:  1.change type,   2.pass as arguments to helper functions,  and  3.add as elements in the value of people_dict
      sex=person[1]
      age=int(person[2])
      respiratory_rate=float(person[3])
      oxygen_saturation=float(person[4])
      heart_rate=float(person[5])
      systolic_bp=float(person[6])
      temperature=float(person[7])
      alertness=person[8]
      inspired_oxygen=person[9]
      performance_status=person[10]

      #using helper functions created previously to get hospital address and name for each patient
      risk=priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_bp, temperature, alertness, inspired_oxygen, performance_status)
      hospital=find_hospital(age,sex,risk)
      address=get_address(hospital)

      people_dict[person[0]] = [sex,age,respiratory_rate,oxygen_saturation,heart_rate,systolic_bp,temperature,alertness,inspired_oxygen,performance_status, risk, hospital, address]

    return people_dict

In [11]:
process_people('https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh')

{'E9559': ['FEMALE',
  40,
  24.0,
  0.96,
  105.0,
  115.0,
  34.9,
  'ALERT',
  'AIR',
  'unrestricted normal activity',
  'Select Specialty Hospital - Northeast Atlanta',
  '1821 CLIFTON ROAD NE'],
 'E9385': ['Female',
  51,
  19.0,
  0.99,
  90.0,
  91.0,
  38.1,
  'ALERT',
  'supplemental oxygen',
  'unrestricted normal activity',
  'Select Specialty Hospital - Northeast Atlanta',
  '1821 CLIFTON ROAD NE'],
 'E3067': ['female',
  40,
  29.0,
  0.96,
  105.0,
  95.0,
  38.1,
  'ALERT',
  'air',
  'unrestricted normal activity',
  'Select Specialty Hospital - Northeast Atlanta',
  '1821 CLIFTON ROAD NE'],
 'E9422': ['FEMALE',
  66,
  19.0,
  0.96,
  135.0,
  115.0,
  38.1,
  'ALERT',
  'AIR',
  'bed/chair bound, no self-care',
  'Wesley Woods Geriatric Hospital',
  '1821 CLIFTON ROAD, NE'],
 'E8661': ['MALE',
  18,
  22.0,
  0.95,
  105.0,
  115.0,
  36.3,
  'Confused or Not Alert',
  'air',
  'unrestricted normal activity',
  'Southwest Hospital and Medical Center',
  '501 FAIRBURN

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [14]:
import requests
def result_checker():
  json_string = requests.get('https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5')
  if json_string.status_code == 200:
    patients = json.loads(json_string.text)
    if (patients==process_people('https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh')):
      print("The results are matching")
    else:
      print("The results do not match")
  else:
    print("Error:", json_string.status_code)

result_checker()

The results are matching


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---